In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection

import pandas as pd

import psycopg2

import sys
sys.path.append('../../')
from utils import get_config

sys.path.append('../')
from gis_resources import san_diego_county_zips,  execute_sql, create_where_clause, read_exact_food_biz_categories, read_exact_unhealthy_food_biz_categories

In [2]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)

Definition a function which takes in below variables and give an opportunity score.

In [3]:
consumer_spending_layer = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a2afc4/FeatureServer/0")
consumer_spending_layer

<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a2afc4/FeatureServer/0">

In [4]:
demographics_layer_1 = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/aff1b6/FeatureServer/0")
demographics_layer_1

<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/aff1b6/FeatureServer/0">

In [5]:
demographics_layer_2 = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/ab8ff7/FeatureServer/0")
demographics_layer_2

<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/ab8ff7/FeatureServer/0">

In [6]:
demographics_layer_1

<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/aff1b6/FeatureServer/0">

In [7]:
# Picking up sample variables for just poc on rendering maps as per our need.
out_fields = ["fips", "X1130_A", "X1132_A", "X1137_A", "X1142_A", "X1147_A", 
                      "X1130FY_A", "X1132FY_A", "X1137FY_A", "X1142FY_A", "X1147FY_A", ]

In [8]:
away_food_spending_df = consumer_spending_layer.query(out_fields=out_fields,
                              as_df=True,
                              return_geometry=False)
away_food_spending_df

,FID,fips,x1130_a,x1130fy_a,x1132_a,x1132fy_a,x1137_a,x1137fy_a,x1142_a,x1142fy_a,x1147_a,x1147fy_a
0,1,060730100101,2942.05,3623.67,858.36,1057.22,1392.62,1715.26,250.6,308.66,303.81,374.19
1,2,060730100102,3457.18,4231.38,1140.71,1396.15,1579.12,1932.75,266.28,325.91,364.58,446.23
2,3,060730100103,2935.24,3277.84,968.49,1081.53,1340.71,1497.2,226.08,252.46,309.54,345.67
3,4,060730100111,4519.91,5287.83,1318.7,1542.75,2139.5,2502.99,385.0,450.41,466.74,546.04
4,5,060730150012,4781.76,6792.55,1459.8,2073.67,2256.23,3205.0,386.28,548.71,475.28,675.15
...,...,...,...,...,...,...,...,...,...,...,...,...
2052,2053,060730221012,8544.28,9033.19,2484.53,2626.69,4098.08,4332.57,702.16,742.34,826.65,873.95
2053,2054,060730221021,9568.42,11358.21,2782.33,3302.77,4589.28,5447.71,786.32,933.4,925.73,1098.89
2054,2055,060730221022,9613.37,11373.74,2795.4,3307.29,4610.84,5455.16,790.02,934.68,930.08,1100.39
2055,2056,060730221023,6183.54,7345.41,1856.83,2205.72,2940.3,3492.77,515.72,612.62,605.01,718.69


##### This is painful: Since we created multiple feature layers, we would always be uncertain as to which variables are present where ?

In [9]:
# Checking TOTPOP_CY variable in 1st layer 
demo_df = demographics_layer_1.query(out_fields=['fips', 'TOTPOP_CY', # 2021 Total Population (Esri)
                                                ],
                                     as_df=True,
                                     return_geometry=False
                                    )
demo_df

,FID,fips,totpop_cy
0,1,060730001001,1199.0
1,2,060730001002,1692.0
2,3,060730002011,902.0
3,4,060730002021,1283.0
4,5,060730002022,911.0
...,...,...,...
2052,2053,060730221012,450.0
2053,2054,060730221021,1802.0
2054,2055,060730221022,1906.0
2055,2056,060730221023,2171.0


In [10]:
# Checking HHPOP_CY, TOTHH_CY variable in 2nd layer 
demo_df_2 = demographics_layer_2.query(out_fields=['fips',
                                                 #'TOTPOP_CY', # 2021 Total Population (Esri)
                                                 'HHPOP_CY', # 2021 Household Population (Esri)
                                                 'TOTHH_CY', # 2021 Total Households (Esri)
                                                ],
                                     as_df=True,
                                     return_geometry=False
                                    )
demo_df_2

,FID,fips,hhpop_cy,tothh_cy
0,1,060730083111,1744.0,631.0
1,2,060730083112,1140.0,449.0
2,3,060730083121,596.0,250.0
3,4,060730083122,1190.0,517.0
4,5,060730083123,781.0,358.0
...,...,...,...,...
2052,2053,060730221012,450.0,184.0
2053,2054,060730221021,1802.0,686.0
2054,2055,060730221022,1906.0,775.0
2055,2056,060730221023,2171.0,666.0


Picking up the overlayed layer created to ger businesses associated with block groups

In [11]:
block_grp_zip_food_biz_overlay_features = FeatureLayer(gis = gis,
                                                       url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/San_Diego_Food_Business_Per_Block_Group/FeatureServer/0")

Calculate Count of food businesses per block group

In [12]:
food_biz_df = block_grp_zip_food_biz_overlay_features.query(out_fields=['FIPS','biz_name','biz_type','Join_Count'],
                                             as_df=True,
                                             return_geometry=False)
food_biz_df

,FIPS,Join_Count,OBJECTID_1,biz_name,biz_type
0,060730208061,1,1,Amici Pizza Pasta & Subs,Italian restaurant;Restaurant
1,060730004001,1,2,Mess Royale,Bagel shop;Restaurant;Sandwich shop
2,060730004001,1,3,Fiesta Cantina,Mexican restaurant;Bar;Bar & grill;Brunch rest...
3,060730004001,1,4,Creme De La Crepe,French restaurant
4,060730121011,1,5,"W.D. Dickinson: Farm, House, Mercantile",Farm;Event venue;Historical landmark
...,...,...,...,...,...
11508,060730083462,1,11509,Chick-fil-A,Fast food restaurant;Breakfast restaurant;Cate...
11509,060730083591,1,11510,Tasty Pot,Hot pot restaurant
11510,060730173061,1,11511,Starbucks,Coffee shop;Breakfast restaurant;Cafe;Coffee s...
11511,060730162021,1,11512,Fuerte's NY Pizzeria,Pizza restaurant;Bar;Beer store;Chicken wings ...


In [13]:
food_density_biz_df = food_biz_df.groupby('FIPS').size().reset_index(name='count_biz')
food_density_biz_df

,FIPS,count_biz
0,060250123012,2
1,060730001002,8
2,060730002011,3
3,060730002012,17
4,060730002021,10
...,...,...
1241,060730221011,1
1242,060730221012,47
1243,060730221022,13
1244,060730221023,1


In [14]:
## Let's Join the consumer spending, demographics and food density
merged_df = pd.merge(away_food_spending_df, demo_df, on='fips', how='inner')
merged_df = pd.merge(merged_df, demo_df_2, on='fips', how='inner')
merged_df = pd.merge(merged_df, food_density_biz_df, left_on='fips', right_on='FIPS', how='left')
merged_df

,FID_x,fips,x1130_a,x1130fy_a,x1132_a,x1132fy_a,x1137_a,x1137fy_a,x1142_a,x1142fy_a,x1147_a,x1147fy_a,FID_y,totpop_cy,FID,hhpop_cy,tothh_cy,FIPS,count_biz
0,1,060730100101,2942.05,3623.67,858.36,1057.22,1392.62,1715.26,250.6,308.66,303.81,374.19,76,1690.0,161,1690.0,500.0,<NA>,NaN
1,2,060730100102,3457.18,4231.38,1140.71,1396.15,1579.12,1932.75,266.28,325.91,364.58,446.23,406,2185.0,405,2185.0,682.0,060730100102,5.0
2,3,060730100103,2935.24,3277.84,968.49,1081.53,1340.71,1497.2,226.08,252.46,309.54,345.67,931,1565.0,533,1565.0,493.0,060730100103,5.0
3,4,060730100111,4519.91,5287.83,1318.7,1542.75,2139.5,2502.99,385.0,450.41,466.74,546.04,932,1485.0,535,1471.0,364.0,060730100111,2.0
4,5,060730150012,4781.76,6792.55,1459.8,2073.67,2256.23,3205.0,386.28,548.71,475.28,675.15,28,1563.0,68,1558.0,625.0,060730150012,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,2053,060730221012,8544.28,9033.19,2484.53,2626.69,4098.08,4332.57,702.16,742.34,826.65,873.95,2053,450.0,2053,450.0,184.0,060730221012,47.0
2053,2054,060730221021,9568.42,11358.21,2782.33,3302.77,4589.28,5447.71,786.32,933.4,925.73,1098.89,2054,1802.0,2054,1802.0,686.0,<NA>,NaN
2054,2055,060730221022,9613.37,11373.74,2795.4,3307.29,4610.84,5455.16,790.02,934.68,930.08,1100.39,2055,1906.0,2055,1906.0,775.0,060730221022,13.0
2055,2056,060730221023,6183.54,7345.41,1856.83,2205.72,2940.3,3492.77,515.72,612.62,605.01,718.69,2056,2171.0,2056,2171.0,666.0,060730221023,1.0


In [15]:
## finding float columns
float_cols = merged_df.select_dtypes(include=['float']).columns
merged_df[float_cols] = merged_df[float_cols].fillna(0)

In [16]:
def calculate_competition(count_food_biz, totpop_cy):
    '''
    Return a competition score.
    '''
    # for now lets just return the difference
    
    try:
        score = count_food_biz/totpop_cy
    except ZeroDivisionError:
        score = 0.0
        
    return score

In [17]:
def calculate_opportunity(count_food_biz,
                           X1130_A # 2022 Food Away from Home 
#                           X1132_A, # 2022 Lunch Away from Home
#                           X1137_A, # 2022 Dinner Away from Home
#                           X1142_A, # 2022 Snacks & Beverages Away from Home
#                           X1147_A, # 2022 Breakfast and Brunch Away from Home
#                           X1130FY_A, 
#                           X1132FY_A, 
#                           X1137FY_A, 
#                           X1142FY_A, 
#                           X1147FY_A,
                          ):
    '''
    Return opportunity score.
    '''
    # All these _A variables are spending per household
        
    try:
        score = X1130_A/count_food_biz
    except ZeroDivisionError:
        score = 0.0
  
    return score
    

In [18]:
merged_df['competition_score'] = merged_df.apply(lambda x: calculate_competition(x.count_biz, x.totpop_cy), axis=1)
merged_df['opportunity_score'] = merged_df.apply(lambda x: calculate_opportunity(x.count_biz, x.x1130_a), axis=1)
merged_df

,FID_x,fips,x1130_a,x1130fy_a,x1132_a,x1132fy_a,x1137_a,x1137fy_a,x1142_a,x1142fy_a,...,x1147fy_a,FID_y,totpop_cy,FID,hhpop_cy,tothh_cy,FIPS,count_biz,competition_score,opportunity_score
0,1,060730100101,2942.05,3623.67,858.36,1057.22,1392.62,1715.26,250.6,308.66,...,374.19,76,1690.0,161,1690.0,500.0,<NA>,0.0,0.000000,0.000000
1,2,060730100102,3457.18,4231.38,1140.71,1396.15,1579.12,1932.75,266.28,325.91,...,446.23,406,2185.0,405,2185.0,682.0,060730100102,5.0,0.002288,691.436000
2,3,060730100103,2935.24,3277.84,968.49,1081.53,1340.71,1497.2,226.08,252.46,...,345.67,931,1565.0,533,1565.0,493.0,060730100103,5.0,0.003195,587.048000
3,4,060730100111,4519.91,5287.83,1318.7,1542.75,2139.5,2502.99,385.0,450.41,...,546.04,932,1485.0,535,1471.0,364.0,060730100111,2.0,0.001347,2259.955000
4,5,060730150012,4781.76,6792.55,1459.8,2073.67,2256.23,3205.0,386.28,548.71,...,675.15,28,1563.0,68,1558.0,625.0,060730150012,8.0,0.005118,597.720000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,2053,060730221012,8544.28,9033.19,2484.53,2626.69,4098.08,4332.57,702.16,742.34,...,873.95,2053,450.0,2053,450.0,184.0,060730221012,47.0,0.104444,181.793191
2053,2054,060730221021,9568.42,11358.21,2782.33,3302.77,4589.28,5447.71,786.32,933.4,...,1098.89,2054,1802.0,2054,1802.0,686.0,<NA>,0.0,0.000000,0.000000
2054,2055,060730221022,9613.37,11373.74,2795.4,3307.29,4610.84,5455.16,790.02,934.68,...,1100.39,2055,1906.0,2055,1906.0,775.0,060730221022,13.0,0.006821,739.490000
2055,2056,060730221023,6183.54,7345.41,1856.83,2205.72,2940.3,3492.77,515.72,612.62,...,718.69,2056,2171.0,2056,2171.0,666.0,060730221023,1.0,0.000461,6183.540000
